# Assignment 1-2: Support Vector Machine

## Multiclass Support Vector Machine exercise


In this exercise you will:
    
- implement a fully-vectorized **loss function** for the SVM
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** using numerical gradient


## CIFAR-10 Data Loading and Preprocessing

# Libraries requre for run the program


In [3]:
pip install tensorflow


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ed/b6/62345568cd07de5d9254fcf64d7e44aacbb6abde11ea953b3cb320e58d19/tensorflow-2.17.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.17.0 from https://files.pythonhosted.org/packages/66/03/5c447feceb72f5a38ac2aa79d306fa5b5772f982c2b480c1329c7e382900/tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/p

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings 
warnings.filterwarnings('ignore')

In [5]:
# Load the data &Extract the data from Cifar 10
import os
import urllib.request
import tarfile
import pickle
import numpy as np

# Function to show download progress
def show_progress(count, block_size, total_size):
    pct_complete = float(count * block_size) / total_size
    msg = "\r- Download progress: {0:.1%}".format(pct_complete)
    print(msg, end='')

# Function to download and extract the dataset
def download_and_unpack(url, target_dir):
    """
    Downloads and extracts the CIFAR-10 dataset if not already present.

    :param url: The URL of the file to download.
    :param target_dir: The directory where the file will be saved and extracted.
    """
    file_name = url.split('/')[-1]
    file_path = os.path.join(target_dir, file_name)

    if not os.path.exists(file_path):
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        print(f"Downloading {file_name}...")
        file_path, _ = urllib.request.urlretrieve(url=url, filename=file_path, reporthook=show_progress)
        print("\nDownload completed. Extracting files...")

        if file_path.endswith(".tar.gz"):
            with tarfile.open(name=file_path, mode="r:gz") as tar_ref:
                tar_ref.extractall(target_dir)
        print("Extraction done.")
    else:
        print("File already downloaded and extracted.")

# Function to load CIFAR-10 batch data
def load_batch(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
        X = data_dict[b'data']
        Y = data_dict[b'labels']
        X = X.reshape(10000, 3, 32, 32).astype("float")
        Y = np.array(Y)
    return X, Y

# Function to load the CIFAR-10 dataset
def load_cifar10(data_dir):
    """
    Loads the entire CIFAR-10 dataset from the given directory.

    :param data_dir: The directory where the CIFAR-10 data is located.
    :return: Tuple of numpy arrays (X_train, Y_train), (X_test, Y_test)
    """
    # Load all the training batches
    X_train = []
    Y_train = []
    for i in range(1, 6):
        batch_file = os.path.join(data_dir, 'data_batch_' + str(i))
        X, Y = load_batch(batch_file)
        X_train.append(X)
        Y_train.append(Y)
    X_train = np.concatenate(X_train)
    Y_train = np.concatenate(Y_train)

    # Load the test batch
    X_test, Y_test = load_batch(os.path.join(data_dir, 'test_batch'))

    return (X_train, Y_train), (X_test, Y_test)

# Define URL and download directory
download_dir = r"C:\Users\jasha\OneDrive\cifar-10-batches-py"  # Replace with your directory
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

# Download and extract CIFAR-10 dataset
download_and_unpack(url, download_dir)

# Load the dataset
data_dir = os.path.join(download_dir, 'cifar-10-batches-py')
(X_train, Y_train), (X_test, Y_test) = load_cifar10(data_dir)

# Check the data shapes
print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {Y_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Test labels shape: {Y_test.shape}")


File already downloaded and extracted.
Training data shape: (50000, 3, 32, 32)
Training labels shape: (50000,)
Test data shape: (10000, 3, 32, 32)
Test labels shape: (10000,)


In [6]:
# UPLOAD THE DATA ON ONE DRIVE

# Define the download directory and URL
download_dir = r"C:\Users\jasha\OneDrive\cifar-10-batches-py"
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"  # URL for CIFAR-10 dataset

# Call the function to download and extract
download_and_unpack(url, download_dir)

File already downloaded and extracted.


In [7]:
# Start the  Data preprocessing

#LOAD CIFAR-10 DATASET
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Check the shape of the loaded data
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 232s 1us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [8]:
# Normalize the images to values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Verify the normalization by printing the minimum and maximum values
print(f"Min pixel value in x_train: {x_train.min()}")
print(f"Max pixel value in x_train: {x_train.max()}")


Min pixel value in x_train: 0.0
Max pixel value in x_train: 1.0


In [9]:
# Performing the one _hot encoding
from tensorflow.keras.utils import to_categorical 

# One-hot encode the labels for training and test datasets
y_train = to_categorical(Y_train, num_classes=10)  
y_test = to_categorical(Y_test, num_classes=10)   

# Check the shape of the one-hot encoded labels
print(f"y_train shape after one-hot encoding: {y_train.shape}")  
print(f"Example of one-hot encoded label: {y_train[0]}")  





y_train shape after one-hot encoding: (50000, 10)
Example of one-hot encoded label: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [10]:
# create a small development set as a subset of the training data;
import numpy as np
from tensorflow.keras.datasets import cifar10

# Load the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Define the number of samples for each subset
num_training = 49000
num_validation = 1000
num_test = 1000
num_dev = 500

# Split the original training data into training and validation sets
# Validation set: from num_training to num_training + num_validation
mask = range(num_training, num_training + num_validation)
X_val = X_train[mask]
y_val = y_train[mask]

# Training set: first num_training points
mask = range(num_training)
X_train = X_train[mask]
y_train = y_train[mask]

# Development set: random subset of training data (500 samples)
mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = X_train[mask]
y_dev = y_train[mask]

# Test set: first num_test points from the original test set
mask = range(num_test)
X_test = X_test[mask]
y_test = y_test[mask]

# Print the shapes of the datasets to verify the splits
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('Dev data shape: ', X_dev.shape)
print('Dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000, 1)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000, 1)
Test data shape:  (1000, 32, 32, 3)
Test labels shape:  (1000, 1)
Dev data shape:  (500, 32, 32, 3)
Dev labels shape:  (500, 1)
